In [22]:
import numpy as np
import pandas as pd
from gnews import GNews
import yfinance as yf
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.parse import urlparse
from gnews import GNews
import time
import json
from scrapping_articles_fun import extract_news_fixed


In [ ]:
def align_with_index(df1, df2, date_col='Date', join_type='inner'):
    """
    Align datasets using pandas align method.
    join_type: 'inner', 'outer', 'left', 'right'
    """
    # Set date as index
    df1_indexed = df1.set_index(date_col)
    df2_indexed = df2.set_index(date_col)
    
    # Align the dataframes
    df1_aligned, df2_aligned = df1_indexed.align(df2_indexed, join=join_type, axis=0)
    
    return df1_aligned.reset_index(), df2_aligned.reset_index()


In [ ]:
df_oat = pd.read_csv("data\\Rendement de l'Obligation France 10 ans - Données Historiques (1).csv")
df_bund = pd.read_csv("data\\Rendement de l'Obligation Allemagne 10 ans - Données Historiques.csv")
df_oat_aligned, df_bund_aligned = align_with_index(df_oat, df_bund, date_col='Date', join_type='inner')
df_oat_aligned['Dernier'] = df_oat_aligned['Dernier'].str.replace(',', '.').astype(float)
df_bund_aligned['Dernier'] = df_bund_aligned['Dernier'].str.replace(',', '.').astype(float)

df_oat_aligned['Date'] = pd.to_datetime(df_oat_aligned['Date'], format='%d/%m/%Y')
df_bund_aligned['Date'] = pd.to_datetime(df_bund_aligned['Date'], format='%d/%m/%Y')

In [ ]:
y = df_oat_aligned['Dernier']/df_bund_aligned['Dernier']

In [ ]:
numbers_of_points = 700
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(df_oat_aligned['Date'][:numbers_of_points], y[:numbers_of_points], label='Ratio OAT/Bund 10 ans', color='blue')
ax.set_title("Ratio OAT/Bund 10 ans")
ax.set_xlabel("Date")
ax.set_ylabel("Ratio OAT/Bund")
plt.xticks(rotation=45)
plt.show()

In [ ]:
#import adf test
from statsmodels.tsa.stattools import adfuller

adf_result = adfuller(y[:numbers_of_points])
print('ADF Statistic:', adf_result[0])
print('p-value:', adf_result[1])

In [ ]:
spread_one_year = y[:numbers_of_points]
spread_one_year_diff = spread_one_year.diff().dropna()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(df_oat_aligned['Date'][1:numbers_of_points], spread_one_year_diff, label='Ratio OAT/Bund 10 ans Diff', color='blue')
ax.set_title("Ratio OAT/Bund 10 ans Diff")
ax.set_xlabel("Date")
ax.set_ylabel("Ratio OAT/Bund Diff")
plt.xticks(rotation=45)
plt.show()

In [ ]:
from fredapi import Fred

# Obtenir une clé API gratuite sur: https://fred.stlouisfed.org/docs/api/api-key.html
FRED_API_KEY = 'c1d79a456b2cc144c9132a39f0865cd8'
fred = Fred(api_key=FRED_API_KEY)

# Dictionnaire des séries FRED
series_dict = {
    'FPCPITOTLZGFRA': 'FR_Inflation_CPI',
    'CLVMNACSCAB1GQFR': 'FR_PIB',
    'LRHUTTTTFRM156S': 'FR_Unemployment_Rate',
    'GGGDTAFRA188N': 'FR_Debt_Rate',
    'FPCPITOTLZGDEU': 'DE_Inflation_CPI',
    'CLVMNACSCAB1GQDE': 'DE_PIB_Growth',
    'LRHUTTTTDEM156S': 'DE_Unemployment_Rate',
    'GGGDTADEA188N': 'DE_Debt_Rate',
}

data_frames = []
for series_id, label in series_dict.items():
    try:
        data = fred.get_series(series_id, observation_start='2020-01-01', observation_end='2026-01-08')
        df = pd.DataFrame({label: data})
        if label == 'FR_PIB' or label == 'DE_PIB_Growth':
            df[label] = (np.log(df[label]/df[label].shift()) * 400).round(2)
        data_frames.append(df)
    except Exception as e:
        print(f"Erreur pour {label}: {e}")

macro_data = pd.concat(data_frames, axis=1)
print(macro_data.head())
# Transformer le PIB en variation trimestrielle


macro_processed = macro_data
macro_processed.to_csv('macro_processed.csv')